## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=2)

# Get the date one week ago
one_week_ago = today - timedelta(days=8)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

June 6 June 14


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,June-11-2025,Condo/Co-op,555 NE 15th St Unit 10J,Miami,FL,33132.0,275000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/555-NE-15th-St...,MARMLS,A11631816,N,Y,25.789865,-80.185704


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-06-11    23
2025-06-10    19
2025-06-09    18
2025-06-12    16
2025-06-13    13
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
79,PAST SALE,2025-06-12,Condo/Co-op,15701 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,4200000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11685302,N,Y,25.921944,-80.121716
73,PAST SALE,2025-06-11,Condo/Co-op,300 Biscayne Blvd Way #1603,Miami,FL,33131.0,4075000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11705387,N,Y,25.770777,-80.187521
72,PAST SALE,2025-06-09,Condo/Co-op,16051 Collins Ave #1804,Sunny Isles Beach,FL,33160.0,3700000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11705314,N,Y,25.926219,-80.121522
60,PAST SALE,2025-06-10,Condo/Co-op,10295 Collins Ave #401,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11714813,N,Y,25.898952,-80.123405
39,PAST SALE,2025-06-13,Condo/Co-op,4701 N Meridian Ave Unit LPH15,Miami Beach,FL,33140.0,3125000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,MARMLS,A11766747,N,Y,25.822813,-80.130733
56,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5204,Miami,FL,33131.0,3050000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11732728,N,Y,25.764264,-80.192725
68,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5608,Miami,FL,33131.0,2550000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11700853,N,Y,25.764264,-80.192725
4,PAST SALE,2025-06-10,Condo/Co-op,2831 S Bayshore Dr #1501,Miami,FL,33133.0,2300000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2831-S-Bayshor...,MARMLS,A11796713,N,Y,25.729356,-80.238554
71,PAST SALE,2025-06-12,Condo/Co-op,400 S Pointe Dr #604,Miami Beach,FL,33139.0,2225000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11712094,N,Y,25.767292,-80.134179
28,PAST SALE,2025-06-12,Condo/Co-op,15901 Collins Ave #2106,Sunny Isles Beach,FL,33160.0,2100000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11783523,N,Y,25.923693,-80.121772


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [16]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [17]:
response_df = pd.DataFrame(response_list)

In [18]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [19]:
df_top_ten = merged_df

## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

89


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$760,808


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$547


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$67,711,900


In [25]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [26]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 Miami-Dade


Input Previous Week Condo Sales Total (number units sold):
123
-----------
Input Previous Week Condo Average Sales Price:
935700
-----------
Input Previous Week Condo Average PSF:
581
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
115091100.0


In [27]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,123,935700,581,115091100.0


In [28]:
last_week['Last Week Total Dollar Volume'].iloc[0]

115091100.0

In [30]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           89              760808          547                     67711900.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [31]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [32]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [33]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [34]:
pd.set_option('display.max_columns',None)

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-12,Condo/Co-op,15701 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,4200000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2673.0,2739.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11685302,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Luz Bencich,"Grand Realty of America, Corp.",None,None,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",None,None,1,orange


In [36]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [37]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-12,Condo/Co-op,15701 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,4200000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2673.0,2739.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11685302,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Luz Bencich,"Grand Realty of America, Corp.",None,None,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",None,None,1,orange
1,PAST SALE,2025-06-11,Condo/Co-op,300 Biscayne Blvd Way #1603,Miami,FL,33131.0,4075000.0,3.0,4.5,Aston Martin Residences,3078.0,NaN,2024.0,NaN,1324.0,5140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11705387,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Sergio Balinotti,"BRG International , LLC",Matias Alem,"BRG International , LLC",Franco Tommasi,The Corcoran Group,None,None,2,blue
2,PAST SALE,2025-06-09,Condo/Co-op,16051 Collins Ave #1804,Sunny Isles Beach,FL,33160.0,3700000.0,3.0,4.5,TURNBERRY OCEAN COLONY NO,3735.0,NaN,2006.0,NaN,991.0,6607.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11705314,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Mitzi Mitchell,Cervera Real Estate Inc.,None,None,Alexandra Javgureanu,Avanti Way Realty LLC,None,None,3,blue
3,PAST SALE,2025-06-10,Condo/Co-op,10295 Collins Ave #401,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,10295 COLLINS AVE RESDNTA,2136.0,NaN,2007.0,NaN,1545.0,1170.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11714813,N,Y,25.898952,-80.123405,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,Jessica Sultan,EXP Realty LLC,None,None,Rosa Noguera,United Realty Group Inc,None,None,4,blue
4,PAST SALE,2025-06-13,Condo/Co-op,4701 N Meridian Ave Unit LPH15,Miami Beach,FL,33140.0,3125000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,1934.0,NaN,2019.0,NaN,1616.0,4003.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,MARMLS,A11766747,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,Darin Tansey,Douglas Elliman,None,None,Darin Tansey,Douglas Elliman,Lynna Weil,Douglas Elliman,5,blue
5,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5204,Miami,FL,33131.0,3050000.0,4.0,4.5,BRICKELL FLATIRON CONDO,2418.0,NaN,2019.0,NaN,1261.0,3867.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11732728,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Michael Wiesenfeld,The Corcoran Group,Anita Freud,The Corcoran Group,Anita Freud,The Corcoran Group,None,None,6,blue
6,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5608,Miami,FL,33131.0,2550000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1809.0,NaN,2019.0,NaN,1410.0,2695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11700853,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Lorena Plaza,EXP Realty LLC,None,None,Evan Schechtman,Blackbook Properties,None,None,7,blue
7,PAST SALE,2025-06-10,Condo/Co-op,2831 S Bayshore Dr #1501,Miami,FL,33133.0,2300000.0,3.0,2.5,CLUB RESIDENCES AT PARK G,1281.0,NaN,2018.0,NaN,1795.0,3500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2831-S-Bayshor...,MARMLS,A11796713,N,Y,25.729356,-80.238554,https://www.redfin.com/FL/Miami/2831-S-Bayshor...,Jill Balli,One Sotheby's International Realty,None,None,Darin Tansey,Douglas Elliman,None,None,8,blue
8,PAST SALE,2025-06-12,Condo/Co-op,400 S Pointe Dr #604,Miami Beach,FL,33139.0,2225000.0,3.0,3.0,SOUTH POINTE TOWERS CONDO,1802.0,NaN,1987.0,NaN,

In [38]:
%store -r map_box_api_key

In [39]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [40]:
m.save('index.html')

## Data snagger

In [41]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [42]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-06-12,Condo/Co-op,15701 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,4200000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2673.0,2739.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11685302,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Luz Bencich,"Grand Realty of America, Corp.",None,None,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [45]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_06162025


# CREATE TEMPLATE 

In [46]:
muni_set = set(df_top_ten['CITY'])

In [47]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [48]:
df_top_ten.reset_index(inplace=True,drop=True)

In [49]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [50]:
top_sale

'15701 COLLINS CONDO at 15701 Collins Ave #3504 in Sunny Isles Beach'

In [51]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-12,Condo/Co-op,15701 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,4200000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2673.0,2739.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11685302,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Luz Bencich,"Grand Realty of America, Corp.",None,None,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",None,None,1,orange,15701 COLLINS CONDO at 15701 Collins Ave #3504...
1,PAST SALE,2025-06-11,Condo/Co-op,300 Biscayne Blvd Way #1603,Miami,FL,33131.0,4075000.0,3.0,4.5,Aston Martin Residences,3078.0,NaN,2024.0,NaN,1324.0,5140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11705387,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Sergio Balinotti,"BRG International , LLC",Matias Alem,"BRG International , LLC",Franco Tommasi,The Corcoran Group,None,None,2,blue,Aston Martin Residences at 300 Biscayne Blvd W...
2,PAST SALE,2025-06-09,Condo/Co-op,16051 Collins Ave #1804,Sunny Isles Beach,FL,33160.0,3700000.0,3.0,4.5,TURNBERRY OCEAN COLONY NO,3735.0,NaN,2006.0,NaN,991.0,6607.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11705314,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Mitzi Mitchell,Cervera Real Estate Inc.,None,None,Alexandra Javgureanu,Avanti Way Realty LLC,None,None,3,blue,TURNBERRY OCEAN COLONY NO at 16051 Collins Ave...
3,PAST SALE,2025-06-10,Condo/Co-op,10295 Collins Ave #401,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,10295 COLLINS AVE RESDNTA,2136.0,NaN,2007.0,NaN,1545.0,1170.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11714813,N,Y,25.898952,-80.123405,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,Jessica Sultan,EXP Realty LLC,None,None,Rosa Noguera,United Realty Group Inc,None,None,4,blue,10295 COLLINS AVE RESDNTA at 10295 Collins Ave...
4,PAST SALE,2025-06-13,Condo/Co-op,4701 N Meridian Ave Unit LPH15,Miami Beach,FL,33140.0,3125000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,1934.0,NaN,2019.0,NaN,1616.0,4003.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,MARMLS,A11766747,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,Darin Tansey,Douglas Elliman,None,None,Darin Tansey,Douglas Elliman,Lynna Weil,Douglas Elliman,5,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
5,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5204,Miami,FL,33131.0,3050000.0,4.0,4.5,BRICKELL FLATIRON CONDO,2418.0,NaN,2019.0,NaN,1261.0,3867.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11732728,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Michael Wiesenfeld,The Corcoran Group,Anita Freud,The Corcoran Group,Anita Freud,The Corcoran Group,None,None,6,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
6,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5608,Miami,FL,33131.0,2550000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1809.0,NaN,2019.0,NaN,1410.0,2695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11700853,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Lorena Plaza,EXP Realty LLC,None,None,Evan Schechtman,Blackbook Properties,None,None,7,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
7,PAST SALE,2025-06-10,Condo/Co-op,2831 S Bayshore Dr #1501,Miami,FL,33133.0,2300000.0,3.0,2.5,CLUB RESIDENCES AT PARK G,1281.0,NaN,2018.0,NaN,1795.0,3500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Mia

In [54]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 15701 COLLINS CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,100,000 to $4,200,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Bal Harbour, Miami, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 89 condo sales totaling $67,711,900 million from June 6 to June 14. The previous week, brokers closed 123 condo sales totaling $115,091,100.

Last week’s units sold for an average of $760,808, lower than the $935,700 average price from the previo

In [55]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [56]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [57]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [58]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-12,Condo/Co-op,15701 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,4200000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2673.0,2739.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11685302,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Luz Bencich,"Grand Realty of America, Corp.",None,None,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",None,None,1,orange,15701 COLLINS CONDO at 15701 Collins Ave #3504...
1,PAST SALE,2025-06-11,Condo/Co-op,300 Biscayne Blvd Way #1603,Miami,FL,33131.0,4075000.0,3.0,4.5,Aston Martin Residences,3078.0,NaN,2024.0,NaN,1324.0,5140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11705387,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Sergio Balinotti,"BRG International , LLC",Matias Alem,"BRG International , LLC",Franco Tommasi,The Corcoran Group,None,None,2,blue,Aston Martin Residences at 300 Biscayne Blvd W...
2,PAST SALE,2025-06-09,Condo/Co-op,16051 Collins Ave #1804,Sunny Isles Beach,FL,33160.0,3700000.0,3.0,4.5,TURNBERRY OCEAN COLONY NO,3735.0,NaN,2006.0,NaN,991.0,6607.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11705314,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Mitzi Mitchell,Cervera Real Estate Inc.,None,None,Alexandra Javgureanu,Avanti Way Realty LLC,None,None,3,blue,TURNBERRY OCEAN COLONY NO at 16051 Collins Ave...
3,PAST SALE,2025-06-10,Condo/Co-op,10295 Collins Ave #401,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,10295 COLLINS AVE RESDNTA,2136.0,NaN,2007.0,NaN,1545.0,1170.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11714813,N,Y,25.898952,-80.123405,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,Jessica Sultan,EXP Realty LLC,None,None,Rosa Noguera,United Realty Group Inc,None,None,4,blue,10295 COLLINS AVE RESDNTA at 10295 Collins Ave...
4,PAST SALE,2025-06-13,Condo/Co-op,4701 N Meridian Ave Unit LPH15,Miami Beach,FL,33140.0,3125000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,1934.0,NaN,2019.0,NaN,1616.0,4003.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,MARMLS,A11766747,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,Darin Tansey,Douglas Elliman,None,None,Darin Tansey,Douglas Elliman,Lynna Weil,Douglas Elliman,5,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
5,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5204,Miami,FL,33131.0,3050000.0,4.0,4.5,BRICKELL FLATIRON CONDO,2418.0,NaN,2019.0,NaN,1261.0,3867.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11732728,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Michael Wiesenfeld,The Corcoran Group,Anita Freud,The Corcoran Group,Anita Freud,The Corcoran Group,None,None,6,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
6,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5608,Miami,FL,33131.0,2550000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1809.0,NaN,2019.0,NaN,1410.0,2695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11700853,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Lorena Plaza,EXP Realty LLC,None,None,Evan Schechtman,Blackbook Properties,None,None,7,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
7,PAST SALE,2025-06-10,Condo/Co-op,2831 S Bayshore Dr #1501,Miami,FL,33133.0,2300000.0,3.0,2.5,CLUB RESIDENCES AT PARK G,1281.0,NaN,2018.0,NaN,1795.0,3500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Mia

In [59]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: 15701 COLLINS CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,100,000 to $4,200,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Bal Harbour, Miami, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 89 condo sales totaling $67,711,900 million from June 6 to June 14. The previous week, brokers closed 123 condo sales totaling $115,091,100.

Last week’s units sold for an average of $760,808, lower than the $935,700 average price from the previo

In [67]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/North-Miami-Beach/15701-Collins-Ave-33160/unit-3504/home/171036238


In [75]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami/300-Biscayne-Boulevard-Way-33131/unit-1603/home/193906478


In [69]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Sunny-Isles-Beach/15901-Collins-Ave-33160/unit-2106/home/42676090


In [70]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/400-S-Pointe-Dr-33139/unit-604/home/43414614


In [71]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/North-Miami-Beach/15701-Collins-Ave-33160/unit-3504/home/171036238


In [72]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/15901-Collins-Ave-33160/unit-2106/home/42676090


In [73]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-06-12,Condo/Co-op,15701 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,4200000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2673.0,2739.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11685302,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Luz Bencich,"Grand Realty of America, Corp.",None,None,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",None,None,1,orange,15701 COLLINS CONDO at 15701 Collins Ave #3504...
1,PAST SALE,2025-06-11,Condo/Co-op,300 Biscayne Blvd Way #1603,Miami,FL,33131.0,4075000.0,3.0,4.5,Aston Martin Residences,3078.0,NaN,2024.0,NaN,1324.0,5140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11705387,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Sergio Balinotti,"BRG International , LLC",Matias Alem,"BRG International , LLC",Franco Tommasi,The Corcoran Group,None,None,2,blue,Aston Martin Residences at 300 Biscayne Blvd W...
2,PAST SALE,2025-06-09,Condo/Co-op,16051 Collins Ave #1804,Sunny Isles Beach,FL,33160.0,3700000.0,3.0,4.5,TURNBERRY OCEAN COLONY NO,3735.0,NaN,2006.0,NaN,991.0,6607.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11705314,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Mitzi Mitchell,Cervera Real Estate Inc.,None,None,Alexandra Javgureanu,Avanti Way Realty LLC,None,None,3,blue,TURNBERRY OCEAN COLONY NO at 16051 Collins Ave...
3,PAST SALE,2025-06-10,Condo/Co-op,10295 Collins Ave #401,Bal Harbour,FL,33154.0,3300000.0,2.0,2.5,10295 COLLINS AVE RESDNTA,2136.0,NaN,2007.0,NaN,1545.0,1170.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11714813,N,Y,25.898952,-80.123405,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,Jessica Sultan,EXP Realty LLC,None,None,Rosa Noguera,United Realty Group Inc,None,None,4,blue,10295 COLLINS AVE RESDNTA at 10295 Collins Ave...
4,PAST SALE,2025-06-13,Condo/Co-op,4701 N Meridian Ave Unit LPH15,Miami Beach,FL,33140.0,3125000.0,2.0,2.5,4701 NORTH MERIDIAN CONDO,1934.0,NaN,2019.0,NaN,1616.0,4003.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,MARMLS,A11766747,N,Y,25.822813,-80.130733,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,Darin Tansey,Douglas Elliman,None,None,Darin Tansey,Douglas Elliman,Lynna Weil,Douglas Elliman,5,blue,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
5,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5204,Miami,FL,33131.0,3050000.0,4.0,4.5,BRICKELL FLATIRON CONDO,2418.0,NaN,2019.0,NaN,1261.0,3867.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11732728,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Michael Wiesenfeld,The Corcoran Group,Anita Freud,The Corcoran Group,Anita Freud,The Corcoran Group,None,None,6,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
6,PAST SALE,2025-06-12,Condo/Co-op,1000 Brickell Plz Ph 5608,Miami,FL,33131.0,2550000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1809.0,NaN,2019.0,NaN,1410.0,2695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11700853,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Lorena Plaza,EXP Realty LLC,None,None,Evan Schechtman,Blackbook Properties,None,None,7,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
7,PAST SALE,2025-06-10,Condo/Co-op,2831 S Bayshore Dr #1501,Miami,FL,33133.0,2300000.0,3.0,2.5,CLUB RESIDENCES AT PARK G,1281.0,NaN,2018.0,NaN,1795.0,3500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Mia

## Time on Market Calculator

In [76]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 12, 13) ## List (Earlier) date
date2 = datetime(2025, 6, 11) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

180


## Clean CSV for Datawrapper Chart

In [77]:
chart_df = df_top_ten

In [78]:
chart_df = chart_df.fillna(" ")

In [79]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [80]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [81]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [93]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [83]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [84]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [85]:
chart_df['PPSF'].astype(float)

0    2673.0
1    1324.0
2     991.0
3    1545.0
4    1616.0
5    1261.0
6    1410.0
7    1795.0
8    1235.0
9     902.0
Name: PPSF, dtype: float64

In [86]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [87]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [88]:
chart_df

,SOLD DATE,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-06-12,15701 Collins Ave #3504 in Sunny Isles Beach,"$4,200,000","1,571","$2,673",2020,"Luz Bencich Grand Realty of America, Corp.","Dolores Urdapilleta Urdapilleta Real Estate, L..."
1,2025-06-11,300 Biscayne Blvd Way #1603 in Miami,"$4,075,000","3,078","$1,324",2024,"Sergio Balinotti BRG International , LLC Matia...",Franco Tommasi The Corcoran Group
2,2025-06-09,16051 Collins Ave #1804 in Sunny Isles Beach,"$3,700,000","3,735",$991,2006,Mitzi Mitchell Cervera Real Estate Inc.,Alexandra Javgureanu Avanti Way Realty LLC
3,2025-06-10,10295 Collins Ave #401 in Bal Harbour,"$3,300,000","2,136","$1,545",2007,Jessica Sultan EXP Realty LLC,Rosa Noguera United Realty Group Inc
4,2025-06-13,4701 N Meridian Ave Unit LPH15 in Miami Beach,"$3,125,000","1,934","$1,616",2019,Darin Tansey Douglas Elliman,Darin Tansey Douglas Elliman Lynna Weil Dougla...
5,2025-06-12,1000 Brickell Plz Ph 5204 in Miami,"$3,050,000","2,418","$1,261",2019,Michael Wiesenfeld The Corcoran Group Anita Fr...,Anita Freud The Corcoran Group
6,2025-06-12,1000 Brickell Plz Ph 5608 in Miami,"$2,550,000","1,809","$1,410",2019,Lorena Plaza EXP Realty LLC,Evan Schechtman Blackbook Properties
7,2025-06-10,2831 S Bayshore Dr #1501 in Miami,"$2,300,000","1,281","$1,795",2018,Jill Balli One Sotheby's International Realty,Darin Tansey Douglas Elliman
8,2025-06-12,400 S Pointe Dr #604 in Miami Beach,"$2,225,000","1,802","$1,235",1987,Colin Feuling Brown Harris Stevens,Raymond Jacomo Douglas Elliman
9,2025-06-12,15901 Collins Ave #2106 in Sunny Isles Beach,"$2,100,000","2,327",$902,2008,"Marcello Agostini AG Real Estate Advisors, LLC...",Olga Davydova Golden Keys Inc


In [89]:
csv_date_string = today.strftime("%m_%d_%Y")

In [90]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [91]:
chart_df.columns

Index(['SOLD DATE', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'PPSF', 'YEAR BUILT',
       'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')